ANN Final Project
Authors: Caleb Johnson, Gabe Schwartz, Evan Kates
Network Module

In [ ]:
# import data from csv file
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Step 2: Load the data
df = pd.read_csv('data.csv')

# Step 3: Define features (X) and targets (y)
# Rookie year features
# feature_cols = ['GP_r', 'MIN_r', 'FG_PCT_r', 'FG3M_r', 'FG3A_r', 'REB_r', 'AST_r', 'PTS_r', 'TOV_r']
feature_cols = ['GP_r', 'MIN_r', 'FG_PCT_r', 'REB_r', 'AST_r', 'PTS_r', 'TOV_r']
# feature_cols = ['GP_r', 'MIN_r', 'FG_PCT_r', 'GS_r', 'REB_r', 'AST_r', 'PTS_r', 'TOV_r']

X = df[feature_cols]

# Sophomore stats to predict
target_cols = ['PTS_s', 'REB_s', 'AST_s']
y = df[target_cols]

# Step 4: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Feature scaling (important for neural nets)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 6: Build the model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],),
                       kernel_regularizer=regularizers.l1(0.001)),  # L1 regularization
    keras.layers.Dense(64, activation='relu',
                       kernel_regularizer=regularizers.l1(0.001)),
    keras.layers.Dense(3)  # Predicting 3 values: PTS, REB, AST
])

# Define early stopping
early_stop = EarlyStopping(
    monitor='val_loss',   # Watch the validation loss
    patience=10,          # Number of epochs to wait after no improvement
    restore_best_weights=True # After stopping, roll back to the best model
)

model.compile(optimizer='adam', loss=losses.Huber(), metrics=['mae'])


/Users/calebjohnson/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [51]:
# Step 7: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=300,          # Big number because early stopping will cut it off early
    batch_size=32,
    callbacks=[early_stop]
)

# Step 8: Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae}")

# Step 9: Make predictions
predictions = model.predict(X_test)

# Show some example predictions
for i in range(5):
    print(f"Predicted: {predictions[i]} | Actual: {y_test.iloc[i].values}")

Epoch 1/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 4.7557 - mae: 4.3996 - val_loss: 4.3542 - val_mae: 3.9953
Epoch 2/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.9400 - mae: 3.5842 - val_loss: 3.6580 - val_mae: 3.3186
Epoch 3/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.2319 - mae: 2.8907 - val_loss: 3.0854 - val_mae: 2.7646
Epoch 4/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.6899 - mae: 2.3669 - val_loss: 2.6092 - val_mae: 2.3059
Epoch 5/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.3877 - mae: 2.0872 - val_loss: 2.2959 - val_mae: 2.0064
Epoch 6/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.1467 - mae: 1.8584 - val_loss: 2.1201 - val_mae: 1.8458
Epoch 7/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.0786 - mae: 1.8130 - val_loss: 1.9727 - val_mae: 1.7261
Epoch 8/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.9482 - mae: 1.7064 - val_loss: 1.9072 - val_mae: 1.6840
Epoch 9/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.78

In [54]:
from scipy import stats

# 1. Predict on test data
y_pred = model.predict(X_test)

# 2. Calculate residuals
errors = y_test.values - y_pred

# 3. Estimate standard deviation of residuals
std_dev = np.std(errors, axis=0)

# 4. Compute 95% confidence intervals
# 1.96 standard deviations covers ~95% for a normal distribution

conf_interval = 1.645 * std_dev

print("We are 90% confident that the stats will be within these ranges:")
print(f"Points: ±{conf_interval[0]:.2f}")
print(f"Rebounds: ±{conf_interval[1]:.2f}")
print(f"Assists: ±{conf_interval[2]:.2f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
We are 90% confident that the stats will be within these ranges:
Points: ±4.60
Rebounds: ±2.73
Assists: ±1.27
